In [10]:
import os

In [11]:
os.chdir("../")

In [12]:
%pwd

'/home/tinashe/codes/chicken2'

In [13]:
#initialising the model arguments
from dataclasses import dataclass
from pathlib import Path

@dataclass( frozen=True)

class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path: Path
    updated_base_model_path: Path
    parameter_image_size: float
    parameter_learning_rate: float
    parameter_include_top: bool
    parameter_weights: str
    parameter_clases: int

In [14]:
from classifier.constants import *
from classifier.utils.common_functions import read_yaml, create_dirctories

In [15]:
%pwd

'/home/tinashe/codes/chicken2'

In [16]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            parameters_filepath= PARAMETERS_FILE_PATH):
         
            self.config=read_yaml(config_filepath)
            self.parameters=read_yaml(parameters_filepath)

            create_dirctories([self.config.artifacts_roots])

    def get_prepare_base_model_config(self)-> PrepareBaseModelConfig:
          config=self.config.prepare_base_model

          create_dirctories([config.root_dir])

          prepare_base_model_config=PrepareBaseModelConfig(
                root_dir=Path(config.root_dir),
                base_model_path= Path(config.base_model_path),
                updated_base_model_path=Path(config.updated_base_model_path),
                parameter_image_size=self.parameters.IMAGE_SIZE,
                parameter_learning_rate=self.parameters.LEARNING_RATE,
                parameter_include_top=self.parameters.INCLUDE_TOP,
                parameter_weights=self.parameters.WEIGHTS,
                parameter_clases=self.parameters.CLASSES
          )

          return prepare_base_model_config
    



In [17]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf


In [18]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.parameter_image_size,
            weights=self.config.parameter_weights,
            include_top=self.config.parameter_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):

        if freeze_all:
            for layer in model.layers:
                model.trainable= False

        elif(freeze_till is not None) and freeze_till>0:
            model.trainable= False


        flatten_in=tf.keras.layers.Flatten()(model.output)
        prediction=tf.keras.layers.Dense(
            units=classes,
            activation= "softmax"
        )(flatten_in)

        full_model=tf.keras.models.Model(
            inputs=model.input,
            output=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizer.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()

        return full_model
    
    def update_base_model(self):
        self.full_model= self._prepare_full_model(
            model=self.model,
            classes=self.config.parameter_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.parameter_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
    
    

